In [4]:
!pip install kagglehub
!pip freeze > requirements.txt




[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iarunava/cell-images-for-detecting-malaria")

print("Path to dataset files:", path)

/Users/ingeborgskaareolsson/Aprendizaje_de_maquinas/MachineLearningViT/vit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 675M/675M [01:12<00:00, 9.83MB/s] 

Extracting files...


Path to dataset files: /Users/ingeborgskaareolsson/.cache/kagglehub/datasets/iarunava/cell-images-for-detecting-malaria/versions/1
